# P5 Vehicle Detection and Tracking

### Import Statements

In [1]:
import cv2
import glob
import time
import pickle
import numpy as np
import matplotlib.pyplot as plt
#import matplotlib.image as mpimg
from skimage.feature import hog
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from scipy.ndimage.measurements import label

from moviepy.editor import VideoFileClip

%matplotlib inline

from lesson_functions import *


## Load car and notcar images

In [ ]:
cars = glob.glob('./images/vehicles/**/*.png', recursive=True)
notcars = glob.glob('./images/non-vehicles/**/*.png', recursive=True)

print('Vehicles:', len(cars))
print('Non-Vehicles:', len(notcars))

# Define parameters

In [2]:
color_space = 'YCrCb' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 9 #32 # HOG orientations
pix_per_cell = 8 #16 # HOG pixels per cell
cell_per_block = 2 # HOG cells per block
hog_channel = 'ALL' # Can be 0, 1, 2, or "ALL"
spatial_size = (32, 32) # Spatial binning dimensions
hist_bins = 32    # Number of histogram bins
spatial_feat = True # Spatial features on or off
hist_feat = True # Histogram features on or off
hog_feat = True # HOG features on or off
y_start_stop = [400, None] # Min and max in y to search in slide_window()

## Function for single_img_features() and for searching windows

In [7]:
# Define a function to extract features from a single image window
# This function is very similar to extract_features()
# just for a single image rather than list of images
def single_img_features(img, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0,
                        spatial_feat=True, hist_feat=True, hog_feat=True):    
    #1) Define an empty list to receive features
    img_features = []
    #2) Apply color conversion if other than 'RGB'
    if color_space != 'RGB':
        if color_space == 'HSV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        elif color_space == 'LUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
        elif color_space == 'HLS':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        elif color_space == 'YUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        elif color_space == 'YCrCb':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    else: feature_image = np.copy(img)      
    #3) Compute spatial features if flag is set
    if spatial_feat == True:
        spatial_features = bin_spatial(feature_image, size=spatial_size)
        #4) Append features to list
        img_features.append(spatial_features)
    #5) Compute histogram features if flag is set
    if hist_feat == True:
        hist_features = color_hist(feature_image, nbins=hist_bins)
        #6) Append features to list
        img_features.append(hist_features)
    #7) Compute HOG features if flag is set
    if hog_feat == True:
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.extend(get_hog_features(feature_image[:,:,channel], 
                                    orient, pix_per_cell, cell_per_block, 
                                    vis=False, feature_vec=True))      
        else:
            hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                        pix_per_cell, cell_per_block, vis=False, feature_vec=True)
        #8) Append features to list
        img_features.append(hog_features)

    #9) Return concatenated array of features
    return np.concatenate(img_features)

# Define a function you will pass an image 
# and the list of windows to be searched (output of slide_windows())
def search_windows(img, windows, clf, scaler, color_space='RGB', 
                    spatial_size=(32, 32), hist_bins=32, 
                    hist_range=(0, 256), orient=9, 
                    pix_per_cell=8, cell_per_block=2, 
                    hog_channel=0, spatial_feat=True, 
                    hist_feat=True, hog_feat=True):

    #1) Create an empty list to receive positive detection windows
    on_windows = []
    #2) Iterate over all windows in the list
    for window in windows:
        #3) Extract the test window from original image
        test_img = cv2.resize(img[window[0][1]:window[1][1], window[0][0]:window[1][0]], (64, 64))      
        #4) Extract features for that window using single_img_features()
        features = single_img_features(test_img, color_space=color_space, 
                            spatial_size=spatial_size, hist_bins=hist_bins, 
                            orient=orient, pix_per_cell=pix_per_cell, 
                            cell_per_block=cell_per_block, 
                            hog_channel=hog_channel, spatial_feat=spatial_feat, 
                            hist_feat=hist_feat, hog_feat=hog_feat)
        #5) Scale extracted features to be fed to classifier
        test_features = scaler.transform(np.array(features).reshape(1, -1))
        #6) Predict using your classifier
        prediction = clf.predict(test_features)
        #7) If positive (prediction == 1) then save the window
        if prediction == 1:
            on_windows.append(window)
    #8) Return windows for positive detections
    return on_windows
    

## Train Classifier and Save Data

In [ ]:
car_features = extract_features(cars, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)
notcar_features = extract_features(notcars, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)

X = np.vstack((car_features, notcar_features)).astype(np.float64)                        
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X)
# Apply the scaler to X
scaled_X = X_scaler.transform(X)

# Define the labels vector
y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))


# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(
    scaled_X, y, test_size=0.2, random_state=rand_state)

print('Using:',orient,'orientations',pix_per_cell,
    'pixels per cell and', cell_per_block,'cells per block')
print('Feature vector length:', len(X_train[0]))
# Use a linear SVC 
svc = LinearSVC()
# Check the training time for the SVC
t=time.time()
svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
# Check the prediction time for a single sample
t=time.time()

print('Save classifier...', end='')
pickle_data = {'X_scaler': X_scaler, 'svc': svc, 'orient': orient, 'pix_per_cell': pix_per_cell, 
               'cell_per_block': cell_per_block, 'spatial_size': spatial_size, 'hist_bins': hist_bins, 
               'color_space': color_space, 'hog_channel': hog_channel, 'spatial_feat': spatial_feat, 
               'hist_feat': hist_feat, 'hog_feat':hog_feat, 'y_start_stop':y_start_stop}
with open('svc_pickle.p', 'wb') as f:
    pickle.dump(pickle_data, f)


## Load Saved Classifier Data

In [3]:
# load classifier
dist_pickle = pickle.load( open("svc_pickle.p", "rb" ) )
svc = dist_pickle["svc"]
X_scaler = dist_pickle["X_scaler"]
orient = dist_pickle["orient"]
pix_per_cell = dist_pickle["pix_per_cell"]
cell_per_block = dist_pickle["cell_per_block"]
spatial_size = dist_pickle["spatial_size"]
hist_bins = dist_pickle["hist_bins"]
#color_space = dist_pickle["color_space"]
#hog_channel = dist_pickle["hog_channel"]
#spatial_feat = dist_pickle["spatial_feat"]
#hist_feat = dist_pickle["hist_feat"]
#hog_feat = dist_pickle["hog_feat"]
#y_start_stop = dist_pickle["y_start_stop"]

## Test Different Sliding Window Search Combinations

In [ ]:
image = mpimg.imread('./test_images/test1.jpg')
draw_image = np.copy(image)
#color_space = 'YCrCb'
#hog_channel = 'ALL'
# Uncomment the following line if you extracted training
# data from .png images (scaled 0 to 1 by mpimg) and the
# image you are searching is a .jpg (scaled 0 to 255)
image = image.astype(np.float32)/255

windows0 = slide_window(image, x_start_stop=[512, None], y_start_stop=(400, 528), 
                    xy_window=(48, 48), xy_overlap=(0.5, 0.5))

hot_windows0 = search_windows(image, windows0, svc, X_scaler, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat) 

windows1 = slide_window(image, x_start_stop=[512, None], y_start_stop=y_start_stop, 
                    xy_window=(96, 96), xy_overlap=(0.5, 0.5))

hot_windows1 = search_windows(image, windows1, svc, X_scaler, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat) 

windows2 = slide_window(image, x_start_stop=[512, None], y_start_stop=(400, 656), 
                    xy_window=(128, 128), xy_overlap=(0.5, 0.5))

hot_windows2 = search_windows(image, windows2, svc, X_scaler, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)

windows3 = slide_window(image, x_start_stop=[512, None], y_start_stop=(400, 694), 
                    xy_window=(192, 192), xy_overlap=(0.5, 0.5))

hot_windows3 = search_windows(image, windows3, svc, X_scaler, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)

window_img = draw_boxes(draw_image, hot_windows0, color=(0, 0, 255), thick=6)
window_img = draw_boxes(window_img, hot_windows1, color=(0, 0, 255), thick=6)
window_img = draw_boxes(window_img, hot_windows2, color=(0, 0, 255), thick=6)
window_img = draw_boxes(window_img, hot_windows3, color=(0, 0, 255), thick=6)

hot_windows = []
hot_windows.append(hot_windows0)
hot_windows.append(hot_windows1)
hot_windows.append(hot_windows2)
hot_windows.append(hot_windows3)

plt.imshow(window_img)
plt.imsave('./output_images/'+'windowed_test6.jpg',window_img)

## Test Heat Mapping an Image

In [ ]:
# Read in image similar to one shown above 
#image = mpimg.imread('./test_image.jpg')
heat = np.zeros_like(image[:,:,0]).astype(np.float)

# Add heat to each box in box list
heat = add_heat(heat,hot_windows)
    
# Apply threshold to help remove false positives
heat = apply_threshold(heat,1)

# Visualize the heatmap when displaying    
heatmap = np.clip(heat, 0, 255)

# Find final boxes from heatmap using label function
labels = label(heatmap)
draw_img = draw_labeled_bboxes(np.copy(image), labels)

fig = plt.figure()
plt.subplot(121)
plt.imshow(draw_img)
plt.imsave('./output_images/'+'labeled_boxes.jpg',draw_img)
plt.title('Car Positions')
plt.subplot(122)
plt.imshow(heatmap, cmap='hot')
plt.imsave('./output_images/'+'heatmap.jpg',heatmap)
plt.title('Heat Map')
fig.tight_layout()

## Final sliding window search function

In [4]:
def findCars(image):
    draw_image = np.copy(image)
    image = image.astype(np.float32)/255

    windows0 = slide_window(image, x_start_stop=[512, None], y_start_stop=(400, 528), 
                        xy_window=(48, 48), xy_overlap=(0.5, 0.5))

    hot_windows0 = search_windows(image, windows0, svc, X_scaler, color_space=color_space, 
                            spatial_size=spatial_size, hist_bins=hist_bins, 
                            orient=orient, pix_per_cell=pix_per_cell, 
                            cell_per_block=cell_per_block, 
                            hog_channel=hog_channel, spatial_feat=spatial_feat, 
                            hist_feat=hist_feat, hog_feat=hog_feat) 

    windows1 = slide_window(image, x_start_stop=[512, None], y_start_stop=y_start_stop, 
                        xy_window=(96, 96), xy_overlap=(0.5, 0.5))

    hot_windows1 = search_windows(image, windows1, svc, X_scaler, color_space=color_space, 
                            spatial_size=spatial_size, hist_bins=hist_bins, 
                            orient=orient, pix_per_cell=pix_per_cell, 
                            cell_per_block=cell_per_block, 
                            hog_channel=hog_channel, spatial_feat=spatial_feat, 
                            hist_feat=hist_feat, hog_feat=hog_feat) 

    windows2 = slide_window(image, x_start_stop=[512, None], y_start_stop=(400, 656), 
                        xy_window=(128, 128), xy_overlap=(0.5, 0.5))

    hot_windows2 = search_windows(image, windows2, svc, X_scaler, color_space=color_space, 
                            spatial_size=spatial_size, hist_bins=hist_bins, 
                            orient=orient, pix_per_cell=pix_per_cell, 
                            cell_per_block=cell_per_block, 
                            hog_channel=hog_channel, spatial_feat=spatial_feat, 
                            hist_feat=hist_feat, hog_feat=hog_feat)

    windows3 = slide_window(image, x_start_stop=[512, None], y_start_stop=(400, 694), 
                        xy_window=(192, 192), xy_overlap=(0.5, 0.5))

    hot_windows3 = search_windows(image, windows3, svc, X_scaler, color_space=color_space, 
                            spatial_size=spatial_size, hist_bins=hist_bins, 
                            orient=orient, pix_per_cell=pix_per_cell, 
                            cell_per_block=cell_per_block, 
                            hog_channel=hog_channel, spatial_feat=spatial_feat, 
                            hist_feat=hist_feat, hog_feat=hog_feat)

    hot_windows = []
    hot_windows.append(hot_windows0)
    hot_windows.append(hot_windows1)
    hot_windows.append(hot_windows2)
    hot_windows.append(hot_windows3)
    
    return hot_windows

## Pipe to process video

In [14]:
import collections
last_heatmaps = collections.deque(maxlen=10)

def processImage(img):
    windows = findCars(img)
    heat = np.zeros_like(img[:,:,0]).astype(np.float)
    heat = add_heat(heat,windows)
    heat = apply_threshold(heat,1)
    heatmap = np.clip(heat, 0, 255)
    last_heatmaps.append(heatmap)
    sum_heatmap = np.array(last_heatmaps).sum(axis=0)
    heatmap = np.clip(sum_heatmap, 0, 255)
    labels = label(sum_heatmap)
    draw_img = draw_labeled_bboxes(np.copy(img), labels)
    return draw_img

In [ ]:
## 

In [19]:
input_video = './videos/project_video.mp4'
output_video = './videos/out_project_video.mp4'

clip = VideoFileClip(input_video)#.subclip(10,15)
video_clip = clip.fl_image(processImage)
video_clip.write_videofile(output_video, audio=False)

[MoviePy] >>>> Building video ./videos/out_project_video.mp4
[MoviePy] Writing video ./videos/out_project_video.mp4


100%|█████████▉| 1260/1261 [19:58<00:00,  1.06it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./videos/out_project_video.mp4 

